In [1]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

%matplotlib inline

In [2]:
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold

from sklearn import metrics
import json

import warnings
warnings.filterwarnings("ignore")


sys.path.append('..')
from lib.line_notif import send_message
from lib.utils import reduce_mem_usage, current_time, unpickle, to_pickle
from lib.utils import one_hot_encoder, apply_agg, multi_combine_categorical_feature
from lib.utils import import_data, get_split_indexer 

In [4]:
ls ../submit/v001_012/

oof_1.csv  oof_4.csv         submission_3.csv
oof_2.csv  submission_1.csv  submission_4.csv
oof_3.csv  submission_2.csv  submit_all_v001_012.csv


In [5]:
DATA_VERSION = "v001"
TRIAL_NO = "012"

In [8]:
save_path = Path(f"../processed/{DATA_VERSION}")
model_path = Path(f"../model/{DATA_VERSION}_{TRIAL_NO}")
submit_path = Path(f"../submit/{DATA_VERSION}_{TRIAL_NO}")

print("start loading...")
test  = unpickle(save_path/"test_002.df.pkl", )
print(f"test loaded.")
test.set_index("id", inplace=True)
print(f"test: {test.shape}")
print(test.shape,)
groups = unpickle(save_path/"lbl_molecule_name.pkl", )

start loading...
test loaded.
test: (2505542, 448)
(2505542, 448)


In [10]:
ls -l ../log/v001_012/

total 32
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 5533 Jun  8 22:58 use_cols_1.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 4425 Jun  9 01:00 use_cols_2.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 6622 Jun  9 01:21 use_cols_3.csv
-rw-rw-r-- 1 kenichi.matsui kenichi.matsui 6622 Jun  9 03:37 use_cols_4.csv


In [ ]:
n_folds = 5
iterations = [29000, 18000, 29000, 29000]

importance_list = []
for mol_type, iteration in zip([1,2,3], iterations):
    print(f"mol_type: {mol_type}")
    importance_sub = []
    
    use_cols = pd.read_csv(f"../log/v001_012/use_cols_{mol_type}.csv", index_col=0)["columns"].values
    for fold_ in tqdm(range(n_folds)):
        saved_model_path = f'../model/{DATA_VERSION}_{TRIAL_NO}/booster_{mol_type}_{fold_:02d}_{iteration}.model'
        bst = lgb.Booster(model_file=saved_model_path)
        importance_sub += [pd.DataFrame({"gain":bst.feature_importance(importance_type='gain')},
                                       index=use_cols).sort_index()]
    importance_df = pd.concat(importance_sub, axis=1)
    importance_df.columns = [f"fold_{i}" for i in range(1,6)]
    importance_df["ave"] = importance_df.mean(axis=1)
    importance_df["ratio"] = importance_df["ave"]/importance_df["ave"].sum()
    importance_list += [importance_df.sort_values("ave", ascending=False)]

mol_type: 1


In [ ]:
importance_list[0]